#cleaning

In [1]:
import re
from copy import deepcopy


# core cleaning for a single string

def clean_units_text(s: str) -> str:
    """
    Normalise inch / hertz representations exactly as in Table 1:
      Inch, inches, ″, -inch, inch to inch
      Hertz, hertz, Hz, HZ, hz, -hz to hz

    Also handles attached forms: 47", 47-inch, 240Hz, 240 HZ, etc.
    Then lowercases the entire string.
    """
    if not isinstance(s, str):
        return s

    #  INCH normalisation

    # 0) any number followed by a quote / double-prime to "<num> inch"
    s = re.sub(
        r'(\d+(?:\.\d+)?)\s*["”″]',
        r'\1 inch',
        s
    )

    # number followed by Inch / inches (no or some space)
    s = re.sub(
        r'(\d+(?:\.\d+)?)[ ]*(Inch|INCH|inch|inches|Inches)\b',
        r'\1 inch',
        s
    )

    # number followed by "-inch" variants to "<num> inch"
    s = re.sub(
        r'(\d+(?:\.\d+)?)[ ]*-\s*(Inch|INCH|inch|inches|Inches)\b',
        r'\1 inch',
        s
    )

    # bare "-inch" token to "inch"
    s = re.sub(
        r'-\s*(Inch|INCH|inch|inches|Inches)\b',
        'inch',
        s
    )

    # standalone Inch / inches to inch
    s = re.sub(
        r'\b(Inch|INCH|inch|inches|Inches)\b',
        'inch',
        s
    )

    # standalone quote / double-prime token to inch
    s = re.sub(
        r'\b["”″]\b',
        'inch',
        s
    )

    #  HERTZ normalisation

    # number followed by Hertz / Hz variants to "<num> hz"
    s = re.sub(
        r'(\d+(?:\.\d+)?)[ ]*(Hertz|hertz|Hz|HZ|hz)\b',
        r'\1 hz',
        s
    )

    # number followed by "-hz" to "<num> hz"
    s = re.sub(
        r'(\d+(?:\.\d+)?)[ ]*-\s*(hz|HZ|Hz|hZ)\b',
        r'\1 hz',
        s
    )

    # 3) bare "-hz" token to "hz"
    s = re.sub(
        r'-\s*(hz|HZ|Hz|hZ)\b',
        'hz',
        s
    )

    # 4) standalone Hertz / Hz / hz to hz
    s = re.sub(
        r'\b(Hertz|hertz|Hz|HZ|hz)\b',
        'hz',
        s
    )

    # ---- lower-case everything (final step) ----
    s = s.lower()

    return s


# ---------- cleaning a single product record ----------

def clean_product_record(record: dict) -> dict:
    rec = deepcopy(record)

    if "title" in rec and isinstance(rec["title"], str):
        rec["title"] = clean_units_text(rec["title"])

    if "featuresMap" in rec and isinstance(rec["featuresMap"], dict):
        cleaned_features = {}
        for k, v in rec["featuresMap"].items():
            cleaned_features[k] = clean_units_text(v) if isinstance(v, str) else v
        rec["featuresMap"] = cleaned_features

    return rec


# ---------- cleaning the full TVs-all-merged structure ----------

def clean_tv_dataset(tv_data: dict) -> dict:
    cleaned = {}
    for model_id, records in tv_data.items():
        new_records = []
        for rec in records:
            if isinstance(rec, dict):
                new_records.append(clean_product_record(rec))
            else:
                new_records.append(rec)
        cleaned[model_id] = new_records
    return cleaned


# ---------- helpers for checks ----------

INCH_VARIANT_PATTERNS = [
    r'\bInch\b', r'\bINCH\b', r'\binches\b',
    r'["”″]', r'-inch',
]
HERTZ_VARIANT_PATTERNS = [
    r'\bHertz\b', r'\bhertz\b', r'\bHz\b', r'\bHZ\b', r'-hz',
]


def iter_all_strings(tv_data: dict):
    for model_id, records in tv_data.items():
        for idx, rec in enumerate(records):
            title = rec.get("title")
            if isinstance(title, str):
                yield (model_id, idx, "title", title)
            fmap = rec.get("featuresMap")
            if isinstance(fmap, dict):
                for k, v in fmap.items():
                    if isinstance(v, str):
                        yield (model_id, idx, f"featuresMap[{k}]", v)


def count_pattern_occurrences(tv_data: dict, pattern: str) -> int:
    regex = re.compile(pattern)
    count = 0
    for _, _, _, text in iter_all_strings(tv_data):
        count += len(regex.findall(text))
    return count


def unit_variant_summary(tv_raw: dict, tv_clean: dict):
    print("=== Unit variant summary (before vs after cleaning) ===")
    for pat in INCH_VARIANT_PATTERNS:
        before = count_pattern_occurrences(tv_raw, pat)
        after = count_pattern_occurrences(tv_clean, pat)
        print(f"inch-variant {pat!r}: before={before}, after={after}")

    for pat in HERTZ_VARIANT_PATTERNS:
        before = count_pattern_occurrences(tv_raw, pat)
        after = count_pattern_occurrences(tv_clean, pat)
        print(f"hz-variant   {pat!r}: before={before}, after={after}")

    inch_canonical = count_pattern_occurrences(tv_clean, r'\binch\b')
    hz_canonical = count_pattern_occurrences(tv_clean, r'\bhz\b')
    print(f"canonical 'inch' after cleaning: {inch_canonical}")
    print(f"canonical 'hz'   after cleaning: {hz_canonical}")


def structural_sanity_checks(tv_raw: dict, tv_clean: dict):
    print("\n=== Structural sanity checks ===")
    assert set(tv_raw.keys()) == set(tv_clean.keys()), "modelID sets differ"

    for model_id in tv_raw:
        raw_list = tv_raw[model_id]
        clean_list = tv_clean[model_id]
        assert len(raw_list) == len(clean_list), f"length mismatch for {model_id}"

        for raw_rec, clean_rec in zip(raw_list, clean_list):
            for field in ("modelID", "shop", "url"):
                if field in raw_rec:
                    if raw_rec[field] != clean_rec.get(field):
                        raise AssertionError(f"{field} changed for modelID={model_id}")

# ---------- example I/O usage + checks ----------

if __name__ == "__main__":
    import json
    from pathlib import Path

    path_in = Path("TVs-all-merged.json")
    path_out = Path("TVs-all-merged-cleaned.json")

    with path_in.open("r", encoding="utf-8") as f:
        tv_raw = json.load(f)

    tv_clean = clean_tv_dataset(tv_raw)

    with path_out.open("w", encoding="utf-8") as f:
        json.dump(tv_clean, f, ensure_ascii=False, indent=2)

    structural_sanity_checks(tv_raw, tv_clean)
    unit_variant_summary(tv_raw, tv_clean)





=== Structural sanity checks ===
=== Unit variant summary (before vs after cleaning) ===
inch-variant '\\bInch\\b': before=172, after=0
inch-variant '\\bINCH\\b': before=0, after=0
inch-variant '\\binches\\b': before=605, after=0
inch-variant '["”″]': before=11077, after=2
inch-variant '-inch': before=12, after=0
hz-variant   '\\bHertz\\b': before=0, after=0
hz-variant   '\\bhertz\\b': before=0, after=0
hz-variant   '\\bHz\\b': before=305, after=0
hz-variant   '\\bHZ\\b': before=0, after=0
hz-variant   '-hz': before=0, after=0
canonical 'inch' after cleaning: 11780
canonical 'hz'   after cleaning: 2970


#mw extraction

In [2]:
import re
from typing import List, Dict, Any, Tuple
import numpy as np

# REGEX DEFINITIONS
TITLE_MW_RE = re.compile(
    r"^[a-zA-Z0-9]*((?:[0-9]+[^0-9, ]+)|(?:[^0-9, ]+[0-9]+))[a-zA-Z0-9]*$"
)

VALUE_MW_RE = re.compile(
    r"^(?:\d+(?:\.\d+)?[a-zA-Z]+|\d+(?:\.\d+)?)$"
)

VALUE_TOKEN_RE = re.compile(r"[A-Za-z0-9\.]+")


def extract_title_model_words(title: str) -> set:
    if title is None:
        title = ""

    mw_set: set = set()

    # simple tokenization: split on whitespace, strip punctuation
    for raw_tok in title.split():
        tok = raw_tok.strip(" ,.;:/()[]{}<>\"'“”’+-")
        if not tok:
            continue
        if TITLE_MW_RE.match(tok):
            mw_set.add(tok)

    return mw_set

def extract_value_model_words_from_string(value: str) -> set:
    if value is None:
        value = ""

    mw_set: set = set()
    for tok in VALUE_TOKEN_RE.findall(str(value)):
        tok = tok.strip()
        if not tok:
            continue

        if VALUE_MW_RE.match(tok):
            m_num = re.match(r"^(\d+(?:\.\d+)?)", tok)
            if m_num:
                mw_set.add(m_num.group(1))
    return mw_set


def extract_value_model_words_from_product(features_map: Dict[str, Any]) -> set:
    mw_set: set = set()
    for v in features_map.values():
        mw_set.update(extract_value_model_words_from_string(str(v)))
    return mw_set


#vectorization

In [3]:
import json
from pathlib import Path
from typing import List, Dict, Any, Tuple, Set
import numpy as np
import re
from collections import Counter, defaultdict
from typing import Dict, Any, List, Tuple, Set

# ---------------------------------------------------------------------
# PATHS
# ---------------------------------------------------------------------

CLEANED_PATH      = Path("TVs-all-merged-cleaned.json")  # output of cleaning step
BINARY_OUT_NPZ    = Path("TVs-binary-vectors.npz")
BINARY_OUT_JSON   = Path("TVs-products-with-binary.json")  # optional, for inspection

def build_global_vocab(MW_title: List[str], MW_value: List[str]) -> Tuple[List[str], Dict[str, int]]:
    """
    MW = union(MW_title, MW_value)
    Returns:
      - vocab: list of model words (fixed order)
      - index: dict model_word -> column index
    """
    MW: Set[str] = set(MW_title) | set(MW_value)
    vocab = sorted(MW)
    index = {mw: j for j, mw in enumerate(vocab)}
    return vocab, index

def build_binary_matrix(
    products: List[Dict[str, Any]],
    vocab_index: Dict[str, int]
) -> np.ndarray:
    """
    Build dense binary matrix X of shape (n_products, n_modelwords)
    X[i, j] = 1 if product i contains model word j (in title or value), else 0.
    """
    n_products = len(products)
    n_mw = len(vocab_index)

    X = np.zeros((n_products, n_mw), dtype=np.uint8)

    for i, prod in enumerate(products):
        title_mw = set(prod.get("title_model_words", []) or [])
        value_mw = set(prod.get("value_model_words", []) or [])
        present = title_mw | value_mw

        for mw in present:
            j = vocab_index.get(mw)
            if j is not None:
                X[i, j] = 1

    return X

from collections import Counter

def build_products_title_only_auto_maxdf(
    tv_clean: Dict[str, list],
    min_df: int = 1,
    max_models: int = 5,
) -> Tuple[List[Dict[str, Any]], List[str], int]:
    """
    1) First pass:
       extract raw title_model_words per product
       compute df_title[token] over products
       compute token_models[token] = set of model_ids

    2) Choose max_df as the largest DF among tokens that appear
       in <= max_models distinct model_ids.

    3) Second pass:
       filter per product using min_df and this max_df
       build products and MW_title

    Returns

    products : list of product dicts with: id, model_key, shop, url, title_clean, FeaturesMap_clean, title_model_words (filtered)
    MW_title : sorted list of kept title tokens (global vocab)
    max_df   : auto-chosen max_df value
    """
    raw_products: List[Dict[str, Any]] = []
    df_title = Counter()
    token_models: Dict[str, Set[str]] = defaultdict(set)

    pid = 0

    # first pass: raw tokens + DF + models per token
    for model_id, records in tv_clean.items():
        for rec in records:
            if not isinstance(rec, dict):
                continue

            title_clean = rec.get("title", "") or ""
            fmap_clean  = rec.get("featuresMap", {}) or {}

            mw_title = sorted(extract_title_model_words(title_clean))

            # DF per product (not per occurrence)
            unique_toks = set(mw_title)
            for tok in unique_toks:
                df_title[tok] += 1
                token_models[tok].add(model_id)

            raw_products.append(
                {
                    "id": pid,
                    "model_key": model_id,
                    "shop": rec.get("shop"),
                    "url": rec.get("url"),
                    "title_clean": title_clean,
                    "featuresMap_clean": fmap_clean,
                    "title_model_words": mw_title,   # unfiltered for now
                }
            )
            pid += 1

    # derive max_df from tokens with small model support
    # candidates: tokens that:
    # appear in at least min_df products
    # appear in <= max_models distinct models
    candidate_dfs = []
    for tok, df in df_title.items():
        if df >= min_df and len(token_models[tok]) <= max_models:
            candidate_dfs.append(df)

    max_df = max(candidate_dfs)
    print(max_df)
    #else:
        # fallback: if nothing matches, pick a conservative upper bound
        # e.g. 1% of products
        #n_products = len(raw_products)
        #max_df = max(min_df, int(0.01 * n_products))

    # second pass: filter per product with min_df, max_df
    allowed_title = {
        tok for tok, df in df_title.items()
        if min_df <= df <= max_df
    }

    products: List[Dict[str, Any]] = []
    MW_title_set: Set[str] = set()

    for p in raw_products:
        ft = [tok for tok in p["title_model_words"] if tok in allowed_title]

        p2 = dict(p)
        p2["title_model_words"] = ft
        products.append(p2)

        MW_title_set.update(ft)

    MW_title = sorted(MW_title_set)

    return products, MW_title, max_df

def main():
    # tv_clean must be loaded from CLEANED_PATH before
    with CLEANED_PATH.open("r", encoding="utf-8") as f:
        tv_clean = json.load(f)

    products, MW_title, max_df = build_products_title_only_auto_maxdf(
    tv_clean,
    min_df=2,
    max_models=5,
    )

    # build vocab + X
    vocab = MW_title
    vocab_index = {mw: j for j, mw in enumerate(vocab)}
    X = build_binary_matrix(products, vocab_index)

    product_ids  = np.array([p["id"] for p in products], dtype=int)
    model_ids    = np.array([p["model_key"] for p in products], dtype=object)
    shops        = np.array([p.get("shop") for p in products], dtype=object)
    urls         = np.array([p.get("url") for p in products], dtype=object)

    np.savez_compressed(
        BINARY_OUT_NPZ,
        X=X,
        vocab=np.array(vocab, dtype=object),
        product_ids=product_ids,
        model_ids=model_ids,
        shops=shops,
        urls=urls,
    )

    print(f"#products: {X.shape[0]}")
    print(f"#model words (dim): {X.shape[1]}")


if __name__ == "__main__":
    main()


5
#products: 1624
#model words (dim): 269


#minhashing

In [4]:
"""
MinHash stage.

Input  : TVs-binary-vectors.npz  (output of the model-word / binary-vector step)
Output : TVs-minhash-signatures.npz containing
         - signatures : (num_hashes, n_products) int64
         - a, b, prime: hash function parameters
         - product_ids, model_ids, shops, urls, vocab  (copied through)
"""

from pathlib import Path
import numpy as np


# PATHS + BASIC PARAMETERS
BINARY_NPZ_PATH   = Path("TVs-binary-vectors.npz")
MINHASH_NPZ_PATH  = Path("TVs-minhash-signatures.npz")

NUM_HASHES = 360
RNG_SEED   = 12345


def _next_prime(n: int) -> int:
    """Small helper: return the smallest prime >= n+1."""
    def is_prime(k: int) -> bool:
        if k <= 3:
            return k >= 2
        if k % 2 == 0:
            return False
        i = 3
        while i * i <= k:
            if k % i == 0:
                return False
            i += 2
        return True

    p = max(2, n + 1)
    while not is_prime(p):
        p += 1
    return p

def compute_minhash_signatures(
    X_binary: np.ndarray,
    num_hashes: int,
    rng_seed: int = RNG_SEED,
) -> tuple[np.ndarray, np.ndarray, np.ndarray, int]:
    """
    X_binary : (n_products, n_modelwords) in {0,1}
               rows   to columns in the MinHash slide
               cols   to rows (r) in the MinHash slide
    num_hashes : number of hash functions t

    Returns
    -------
    signatures : (num_hashes, n_products) array
                 column j is the MinHash signature for product j
    a, b       : (num_hashes,) arrays with hash parameters
    prime      : int, modulus used in hash functions
    """
    # Characteristic matrix with rows = model words, cols = products
    C = X_binary.T.astype(np.uint8)          # shape: (n_rows, n_cols)
    n_rows, n_cols = C.shape

    rng = np.random.default_rng(rng_seed)

    prime = _next_prime(n_rows)
    a = rng.integers(1, prime, size=num_hashes, dtype=np.int64)
    b = rng.integers(0, prime, size=num_hashes, dtype=np.int64)

    # Initialise signature matrix with +inf (max int)
    max_int = np.iinfo(np.int64).max
    signatures = np.full((num_hashes, n_cols), max_int, dtype=np.int64)

    # for each row r
    for r in range(n_rows):
        row = C[r, :]
        cols_with_1 = np.nonzero(row)[0]
        if cols_with_1.size == 0:
            continue

        # for each hash function hi compute hi(r)
        # hi(r) = (a_i * r + b_i) mod prime
        r_val = int(r)
        hash_vals = (a * r_val + b) % prime          # shape: (num_hashes,)

        # for each column c that has 1 in row r:
        #   if hi(r) < M(i,c) then M(i,c) := hi(r)
        signatures[:, cols_with_1] = np.minimum(
            signatures[:, cols_with_1],
            hash_vals[:, None],
        )

    return signatures, a, b, prime

def run_minhash_stage(
    binary_npz_path: Path = BINARY_NPZ_PATH,
    out_npz_path: Path = MINHASH_NPZ_PATH,
    num_hashes: int = NUM_HASHES,
    rng_seed: int = RNG_SEED,
) -> None:
    # load binary vectors with metadata (modelID etc.)
    data = np.load(binary_npz_path, allow_pickle=True)

    X          = data["X"]              # (n_products, n_modelwords)
    vocab      = data["vocab"]
    product_ids = data["product_ids"]
    model_ids   = data["model_ids"]
    shops       = data["shops"]
    urls        = data["urls"]

    signatures, a, b, prime = compute_minhash_signatures(
        X_binary=X,
        num_hashes=num_hashes,
        rng_seed=rng_seed,
    )

    # persist signatures + hash params + metadata for later LSH/MSM/eval
    np.savez_compressed(
        out_npz_path,
        signatures=signatures,
        a=a,
        b=b,
        prime=np.array(prime, dtype=np.int64),
        vocab=vocab,
        product_ids=product_ids,
        model_ids=model_ids,   #carried through for later evaluation
        shops=shops,
        urls=urls,
    )

    print(f"[MinHash] #products: {signatures.shape[1]}")
    print(f"[MinHash] #hash functions (signature length): {signatures.shape[0]}")



#lsh

In [5]:
from pathlib import Path
from itertools import combinations
from typing import List, Tuple

import numpy as np

MINHASH_NPZ_PATH = Path("TVs-minhash-signatures.npz")
LSH_OUT_NPZ_PATH = Path("TVs-lsh-candidates.npz")

def all_factor_pairs(n: int) -> List[Tuple[int, int]]:
    """
    All (r, b) with r * b = n.
    r = rows per band, b = number of bands.
    """
    out: List[Tuple[int, int]] = []
    for r in range(1, n + 1):
        if n % r == 0:
            b = n // r
            out.append((r, b))
    return out


def choose_b_r_for_t(
    n_hashes: int,
    target_t: float,
    max_slack: int = 3,
) -> Tuple[int, int, float, int]:
    """
    Given signature length n_hashes and desired threshold t,
    pick (r, b) with r * b = m where m is in [n_hashes - max_slack, n_hashes]
    such that t_hat = (1 / b) ** (1 / r) is closest to target_t.

    Returns
    -------
    r : rows per band
    b : number of bands
    t_hat : resulting effective threshold (1 / b) ** (1 / r)
    used_n_hashes : m actually used (<= n_hashes)
    """
    best_r, best_b, best_t_hat = None, None, None
    best_diff = float("inf")
    best_m = None

    lo = max(1, n_hashes - max_slack)
    hi = n_hashes                      # <-- no + max_slack

    for m in range(lo, hi + 1):
        for r, b in all_factor_pairs(m):
            t_hat = (1.0 / b) ** (1.0 / r)
            diff = abs(t_hat - target_t)
            if diff < best_diff:
                best_diff = diff
                best_r, best_b, best_t_hat, best_m = r, b, t_hat, m

    return best_r, best_b, best_t_hat, best_m

def lsh_candidate_pairs(signatures: np.ndarray, bands: int, rows_per_band: int) -> np.ndarray:
    m, n = signatures.shape
    assert bands * rows_per_band <= m

    buckets = {}

    for band in range(bands):
        start = band * rows_per_band
        end   = start + rows_per_band

        band_block = signatures[start:end, :]  # (rows_per_band, n)

        for j in range(n):
            # IMPORTANT: include band in key
            key = (band, tuple(band_block[:, j].tolist()))
            buckets.setdefault(key, []).append(j)

    # collect candidate pairs (unique)
    pairs = set()
    for idxs in buckets.values():
        if len(idxs) < 2:
            continue
        idxs = sorted(idxs)
        for a in range(len(idxs)):
            ia = idxs[a]
            for b in range(a + 1, len(idxs)):
                ib = idxs[b]
                pairs.add((ia, ib))

    if not pairs:
        return np.empty((0, 2), dtype=int)
    return np.array(sorted(pairs), dtype=int)

def minhash_jaccard(
    signatures: np.ndarray,
    pairs: np.ndarray,
) -> np.ndarray:
    """
    signatures : (num_hashes, n_products)
    pairs      : (K, 2) array of indices (i, j)

    Returns
    -------
    sim : (K,) array with estimated Jaccard similarities
          = fraction of rows where signatures[:, i] == signatures[:, j]
    """
    num_hashes = signatures.shape[0]
    sims = np.empty(pairs.shape[0], dtype=float)

    for idx, (i, j) in enumerate(pairs):
        sims[idx] = np.mean(signatures[:, i] == signatures[:, j])

    return sims

def run_lsh_stage(
    minhash_npz_path: Path = MINHASH_NPZ_PATH,
    out_npz_path: Path = LSH_OUT_NPZ_PATH,
    sim_threshold: float = 0.5,
    max_slack: int = 3,
    bands: int | None = None,
    rows_per_band: int | None = None,
) -> None:
    data = np.load(minhash_npz_path, allow_pickle=True)

    signatures   = data["signatures"]          # (num_hashes, n_products)
    a            = data["a"]
    b_hash       = data["b"]
    prime        = int(data["prime"])
    vocab        = data["vocab"]
    product_ids  = data["product_ids"]
    model_ids    = data["model_ids"]
    shops        = data["shops"]
    urls         = data["urls"]

    num_hashes, n_products = signatures.shape

    # Auto-tune (r, b) if not given
    if bands is None or rows_per_band is None:
        r, b, t_hat, used_m = choose_b_r_for_t(
            n_hashes=num_hashes,
            target_t=sim_threshold,
            max_slack=max_slack,
        )

        if used_m < num_hashes:
            signatures = signatures[:used_m, :]
            num_hashes = used_m

        rows_per_band = r
        bands = b

        print(
            f"[LSH] Auto params for target t={sim_threshold}: "
            f"r={rows_per_band}, b={bands}, t_hat={t_hat:.3f}, m={num_hashes}"
        )
        '''
    else:
        assert bands * rows_per_band == num_hashes, (
            f"bands * rows_per_band = {bands * rows_per_band}, "
            f"but num_hashes = {num_hashes}"
        )
      '''
    t_hat = (1.0 / bands) ** (1.0 / rows_per_band)

    #LSH banding to raw candidate pairs
    pairs = lsh_candidate_pairs(
        signatures,
        bands=bands,
        rows_per_band=rows_per_band,
    )

    # Estimated Jaccard for each candidate
    sims = minhash_jaccard(signatures, pairs)

    # Apply similarity threshold t
    keep_mask = sims >= sim_threshold
    pairs_kept = pairs[keep_mask]
    sims_kept = sims[keep_mask]

    # Map to modelIDs for evaluation
    i_idx = pairs_kept[:, 0]
    j_idx = pairs_kept[:, 1]

    model_ids_i = model_ids[i_idx]
    model_ids_j = model_ids[j_idx]
    same_model  = (model_ids_i == model_ids_j)

    np.savez_compressed(
        out_npz_path,
        i_idx=i_idx,
        j_idx=j_idx,
        sim_est=sims_kept,
        same_model=same_model,
        product_ids=product_ids,
        model_ids=model_ids,
        shops=shops,
        urls=urls,
        vocab=vocab,
        a=a,
        b=b_hash,
        prime=np.array(prime, dtype=np.int64),
        bands=np.array(bands, dtype=int),
        rows_per_band=np.array(rows_per_band, dtype=int),
        sim_threshold=np.array(sim_threshold, dtype=float),
        sim_threshold_effective=np.array(t_hat, dtype=float),
    )

    print(f"[LSH] #products: {n_products}")
    print(f"[LSH] #candidate pairs before threshold: {pairs.shape[0]}")
    print(f"[LSH] #candidate pairs after threshold t={sim_threshold}: {pairs_kept.shape[0]}")


#bootstrap and eval

In [6]:
from sklearn.cluster import AgglomerativeClustering
from pathlib import Path
import numpy as np

# PATHS
BINARY_NPZ_PATH = Path("TVs-binary-vectors.npz")
LSH_NPZ_PATH    = Path("TVs-lsh-candidates.npz")
BOOT_OUT_NPZ    = Path("TVs-bootstrap-eval.npz")

def exact_jaccard_on_candidates(
    X: np.ndarray,
    i_idx: np.ndarray,
    j_idx: np.ndarray,
) -> np.ndarray:
    """
    X     : (n_products, n_features) binary matrix
    i_idx : (K,) candidate indices
    j_idx : (K,) candidate indices

    Returns
    -------
    sims  : (K,) exact Jaccard similarities
    """
    Xi = X[i_idx]  # (K, p)
    Xj = X[j_idx]  # (K, p)

    intersection = np.logical_and(Xi, Xj).sum(axis=1)
    union        = np.logical_or(Xi, Xj).sum(axis=1)

    sims = np.zeros_like(intersection, dtype=float)
    nonzero = union > 0
    sims[nonzero] = intersection[nonzero] / union[nonzero]
    # if union == 0, similarity stays 0
    return sims

def cluster_with_threshold(
    n_items: int,
    i_idx: np.ndarray,
    j_idx: np.ndarray,
    sims: np.ndarray,
    theta: float,
) -> np.ndarray:
    """
    n_items : total number of products
    i_idx, j_idx, sims : candidate edges with exact similarity
    theta   : distance_threshold in
              AgglomerativeClustering with complete linkage.

    Returns
    -------
    labels  : (n_items,) cluster labels
              (complete-linkage agglomerative clustering at similarity >= theta)
    """
    # Convert similarities to distances: d = 1 - s
    d_edges = 1.0 - np.asarray(sims, dtype=float)

    # Build full distance matrix
    dist_matrix = np.ones((n_items, n_items), dtype=float)
    np.fill_diagonal(dist_matrix, 0.0)

    # Fill distances for candidate pairs (symmetric)
    i_idx = np.asarray(i_idx, dtype=int)
    j_idx = np.asarray(j_idx, dtype=int)
    dist_matrix[i_idx, j_idx] = d_edges
    dist_matrix[j_idx, i_idx] = d_edges

    # Map similarity threshold theta to distance threshold
    distance_threshold = 1.0 - float(theta)

    # Agglomerative clustering with complete linkage on the precomputed distances
    agg = AgglomerativeClustering(
        n_clusters=None,
        metric="precomputed",
        linkage="complete",
        distance_threshold=distance_threshold,
        compute_full_tree=True,
    )

    labels = agg.fit_predict(dist_matrix)
    return labels

def pairwise_confusion_and_f1_subset(
    labels: np.ndarray,
    model_ids: np.ndarray,
    subset_idx: np.ndarray,
) -> dict:
    """
    labels     : (n,) cluster labels
    model_ids  : (n,) gold model IDs
    subset_idx : indices to include in evaluation (e.g. OOB set)

    TP = predicted duplicate AND same modelID
    FP = predicted duplicate AND different modelID
    TN = predicted non-duplicate AND different modelID
    FN = predicted non-duplicate AND same modelID

    'predicted duplicate' = same cluster label.
    """
    idx = np.asarray(subset_idx, dtype=int)
    m = idx.size

    TP = FP = TN = FN = 0

    for a in range(m):
        i = idx[a]
        li = labels[i]
        mi = model_ids[i]
        for b in range(a + 1, m):
            j = idx[b]
            same_cluster = (li == labels[j])
            pred_dup = same_cluster
            true_dup = (mi == model_ids[j])

            if pred_dup and true_dup:
                TP += 1
            elif pred_dup and not true_dup:
                FP += 1
            elif (not pred_dup) and true_dup:
                FN += 1
            else:
                TN += 1

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    recall    = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    if precision + recall > 0.0:
        F1 = 2.0 * precision * recall / (precision + recall)
    else:
        F1 = 0.0

    return {
        "TP": TP,
        "FP": FP,
        "TN": TN,
        "FN": FN,
        "precision": precision,
        "recall": recall,
        "F1": F1,
    }

def bootstrap_evaluation(
    labels: np.ndarray,
    model_ids: np.ndarray,
    n_bootstrap: int = 5,
    rng_seed: int = 123,
) -> dict:
    """
    labels, model_ids defined on ALL n items.

    For b = 1..n_bootstrap:
        - draw n indices with replacement => in-bag
        - OOB = {0..n-1} \\ in-bag
        - evaluate F1 only on OOB items (out-of-sample)
    """
    n = len(labels)
    rng = np.random.default_rng(rng_seed)

    F1_list = []
    prec_list = []
    rec_list = []
    TP_list, FP_list, TN_list, FN_list = [], [], [], []

    for b in range(n_bootstrap):
        in_bag = rng.integers(0, n, size=n)
        in_bag_mask = np.zeros(n, dtype=bool)
        in_bag_mask[in_bag] = True

        oob_idx = np.where(~in_bag_mask)[0]

        if oob_idx.size < 2:
            continue

        stats = pairwise_confusion_and_f1_subset(
            labels=labels,
            model_ids=model_ids,
            subset_idx=oob_idx,
        )

        F1_list.append(stats["F1"])
        prec_list.append(stats["precision"])
        rec_list.append(stats["recall"])
        TP_list.append(stats["TP"])
        FP_list.append(stats["FP"])
        TN_list.append(stats["TN"])
        FN_list.append(stats["FN"])

    F1_arr   = np.array(F1_list, dtype=float)
    prec_arr = np.array(prec_list, dtype=float)
    rec_arr  = np.array(rec_list, dtype=float)

    avg_F1  = float(F1_arr.mean()) if F1_arr.size > 0 else 0.0
    std_F1  = float(F1_arr.std(ddof=1)) if F1_arr.size > 1 else 0.0
    avg_pr  = float(prec_arr.mean()) if prec_arr.size > 0 else 0.0
    avg_rec = float(rec_arr.mean()) if rec_arr.size > 0 else 0.0

    return {
        "F1_per_boot": F1_arr,
        "precision_per_boot": prec_arr,
        "recall_per_boot": rec_arr,
        "TP_per_boot": np.array(TP_list, dtype=int),
        "FP_per_boot": np.array(FP_list, dtype=int),
        "TN_per_boot": np.array(TN_list, dtype=int),
        "FN_per_boot": np.array(FN_list, dtype=int),
        "avg_F1": avg_F1,
        "std_F1": std_F1,
        "avg_precision": avg_pr,
        "avg_recall": avg_rec,
        "n_effective_bootstrap": F1_arr.size,
    }

def run_bootstrap_stage(
    binary_npz_path: Path = BINARY_NPZ_PATH,
    lsh_npz_path: Path = LSH_NPZ_PATH,
    out_npz_path: Path = BOOT_OUT_NPZ,
    theta: float = 0.9,
    n_bootstrap: int = 10,
    rng_seed: int = 123,
) -> None:
    bin_data = np.load(binary_npz_path, allow_pickle=True)
    X           = bin_data["X"].astype(bool)
    vocab       = bin_data["vocab"]
    product_ids = bin_data["product_ids"]
    model_ids   = bin_data["model_ids"]
    shops       = bin_data["shops"]
    urls        = bin_data["urls"]

    n_items = X.shape[0]

    lsh_data = np.load(lsh_npz_path, allow_pickle=True)
    i_idx = lsh_data["i_idx"]
    j_idx = lsh_data["j_idx"]

    sims_exact = exact_jaccard_on_candidates(X, i_idx, j_idx)

    labels = cluster_with_threshold(
        n_items=n_items,
        i_idx=i_idx,
        j_idx=j_idx,
        sims=sims_exact,
        theta=theta,
    )

    boot_res = bootstrap_evaluation(
        labels=labels,
        model_ids=model_ids,
        n_bootstrap=n_bootstrap,
        rng_seed=rng_seed,
    )

    np.savez_compressed(
        out_npz_path,
        theta=np.array(theta, dtype=float),
        n_bootstrap=np.array(n_bootstrap, dtype=int),
        labels=labels,
        product_ids=product_ids,
        model_ids=model_ids,
        shops=shops,
        urls=urls,
        vocab=vocab,
        F1_per_boot=boot_res["F1_per_boot"],
        precision_per_boot=boot_res["precision_per_boot"],
        recall_per_boot=boot_res["recall_per_boot"],
        TP_per_boot=boot_res["TP_per_boot"],
        FP_per_boot=boot_res["FP_per_boot"],
        TN_per_boot=boot_res["TN_per_boot"],
        FN_per_boot=boot_res["FN_per_boot"],
        avg_F1=np.array(boot_res["avg_F1"], dtype=float),
        std_F1=np.array(boot_res["std_F1"], dtype=float),
        avg_precision=np.array(boot_res["avg_precision"], dtype=float),
        avg_recall=np.array(boot_res["avg_recall"], dtype=float),
        n_effective_bootstrap=np.array(
            boot_res["n_effective_bootstrap"], dtype=int
        ),
    )

    print(f"[Bootstrap] #products: {n_items}")
    print(f"[Bootstrap] theta (exact Jaccard): {theta}")
    print(f"[Bootstrap] requested bootstraps: {n_bootstrap}")
    print(f"[Bootstrap] effective bootstraps (non-empty OOB): {boot_res['n_effective_bootstrap']}")
    print(f"[Bootstrap] avg F1 (OOB): {boot_res['avg_F1']:.4f} ± {boot_res['std_F1']:.4f}")
    print(f"[Bootstrap] avg precision (OOB): {boot_res['avg_precision']:.4f}")
    print(f"[Bootstrap] avg recall (OOB): {boot_res['avg_recall']:.4f}")


#main block

In [7]:
def run_full_pipeline(
    # MinHash parameters
    minhash_rng_seed: int = 12345,

    # LSH parameters
    lsh_sim_threshold: float = 0.3,
    lsh_max_slack: int = 3,
    lsh_bands: int | None = None,
    lsh_rows_per_band: int | None = None,

    # Exact-Jaccard clustering + bootstrap parameters
    theta_exact: float = 0.3,
    n_bootstrap: int = 10,
    bootstrap_rng_seed: int = 123,

    # Path overrides if needed
    binary_npz_path: Path = BINARY_NPZ_PATH,
    minhash_npz_path: Path = MINHASH_NPZ_PATH,
    lsh_npz_path: Path = LSH_NPZ_PATH,
    boot_out_npz_path: Path = BOOT_OUT_NPZ,
) -> None:
    """
    Unified driver:

    0) Determine num_hashes as 50% of binary vector dimension.
    1) MinHash signatures from binary vectors.
    2) LSH candidate pairs.
    3) Exact-Jaccard clustering + bootstrap evaluation.
    """


    bin_data = np.load(binary_npz_path, allow_pickle=True)
    X = bin_data["X"]
    dim = X.shape[1]
    num_hashes = 360

    print(f"Binary dimension: {dim}")
    print(f"Using num_hashes = {num_hashes}")

    print("=== Stage 1: MinHash ===")
    run_minhash_stage(
        binary_npz_path=binary_npz_path,
        out_npz_path=minhash_npz_path,
        num_hashes=num_hashes,
        rng_seed=minhash_rng_seed,
    )

    # --- Stage 2: LSH ---
    print("\n=== Stage 2: LSH ===")
    run_lsh_stage(
        minhash_npz_path=minhash_npz_path,
        out_npz_path=lsh_npz_path,
        sim_threshold=lsh_sim_threshold,
        max_slack=lsh_max_slack,
        bands=lsh_bands,
        rows_per_band=lsh_rows_per_band,
    )

    print("\n=== Stage 3: Exact Jaccard + Bootstrap eval ===")
    run_bootstrap_stage(
        binary_npz_path=binary_npz_path,
        lsh_npz_path=lsh_npz_path,
        out_npz_path=boot_out_npz_path,
        theta=theta_exact,
        n_bootstrap=n_bootstrap,
        rng_seed=bootstrap_rng_seed,
    )

if __name__ == "__main__":
    # single place to tweak all experiment parameters
    run_full_pipeline(
        minhash_rng_seed=12345,

        lsh_sim_threshold=0.3,    # LSH similarity threshold on MinHash
        lsh_max_slack=0,
        lsh_bands=None,           # or set explicit integers
        lsh_rows_per_band=None,

        theta_exact=0.3,          # exact Jaccard threshold for clustering
        n_bootstrap=5,
        bootstrap_rng_seed=123,
    )

Binary dimension: 269
Using num_hashes = 360
=== Stage 1: MinHash ===
[MinHash] #products: 1624
[MinHash] #hash functions (signature length): 360

=== Stage 2: LSH ===
[LSH] Auto params for target t=0.3: r=4, b=90, t_hat=0.325, m=360
[LSH] #products: 1624
[LSH] #candidate pairs before threshold: 561661
[LSH] #candidate pairs after threshold t=0.3: 561659

=== Stage 3: Exact Jaccard + Bootstrap eval ===
[Bootstrap] #products: 1624
[Bootstrap] theta (exact Jaccard): 0.3
[Bootstrap] requested bootstraps: 5
[Bootstrap] effective bootstraps (non-empty OOB): 5
[Bootstrap] avg F1 (OOB): 0.6297 ± 0.0318
[Bootstrap] avg precision (OOB): 0.6883
[Bootstrap] avg recall (OOB): 0.5820


#plot

In [8]:
import numpy as np
import matplotlib.pyplot as plt

def _fraction_comparisons(n_sub: int, n_cand: int) -> float:
    denom = n_sub * (n_sub - 1) // 2
    return (n_cand / denom) if denom > 0 else 0.0

def all_band_configs(m: int):
    cfgs = []
    for r in range(1, m + 1):
        if m % r == 0:
            b = m // r
            # implied threshold curve midpoint (paper uses this notion)
            # you only need it for labeling/sorting
            t_imp = (1.0 / b) ** (1.0 / r)
            cfgs.append((t_imp, b, r))
    cfgs.sort(key=lambda x: x[0])  # increasing implied threshold
    return cfgs

def run_one_bootstrap_for_t(
    X_bool: np.ndarray,            # (n, p) boolean binary vectors
    signatures: np.ndarray,        # (m, n) MinHash signatures for ALL products
    model_ids_all: np.ndarray,     # (n,) gold labels
    t: float,
    theta_exact: float,
    rng: np.random.Generator,
    max_slack: int = 3,
):

    #One bootstrap replicate for one LSH threshold t

    n = X_bool.shape[0]
    in_bag = rng.integers(0, n, size=n)
    subset_idx = np.unique(in_bag)
    n_sub = subset_idx.size
    if n_sub < 2:
        return 0.0, 0.0

    # subset views
    sig_sub = signatures[:, subset_idx]
    X_sub   = X_bool[subset_idx]
    mid_sub = model_ids_all[subset_idx]

    # choose (r,b) for t
    m_hashes = sig_sub.shape[0]
    r, b, t_hat, used_m = choose_b_r_for_t(
        n_hashes=m_hashes,
        target_t=float(t),
        max_slack=0,
    )
    if used_m < m_hashes:
        sig_sub = sig_sub[:used_m, :]

    # LSH bucket candidates then filtered by estimated sim >= t
    raw_pairs = lsh_candidate_pairs(sig_sub, bands=b, rows_per_band=r)
    if raw_pairs.shape[0] == 0:
        # no comparisons made so F1 zero
        return 0.0, 0.0

    sim_est = minhash_jaccard(sig_sub, raw_pairs)
    keep = sim_est >= float(t)
    pairs = raw_pairs[keep]
    if pairs.shape[0] == 0:
        return 0.0, 0.0

    frac = _fraction_comparisons(n_sub=n_sub, n_cand=pairs.shape[0])

    # exact Jaccard on candidate pairs
    i_loc = pairs[:, 0]
    j_loc = pairs[:, 1]
    sims_exact = exact_jaccard_on_candidates(X_sub, i_loc, j_loc)

    # clustering on subset
    labels_sub = cluster_with_threshold(
        n_items=n_sub,
        i_idx=i_loc,
        j_idx=j_loc,
        sims=sims_exact,
        theta=theta_exact,
    )

    # evaluate final clustering F1 on subset (pairwise)
    stats = pairwise_confusion_and_f1_subset(
        labels=labels_sub,
        model_ids=mid_sub,
        subset_idx=np.arange(n_sub),
    )
    return float(stats["F1"]), float(frac)

def paper_style_curve_finalF1_vs_fraction(
    binary_npz_path=BINARY_NPZ_PATH,
    minhash_npz_path=MINHASH_NPZ_PATH,
    n_bootstrap: int = 100,
    rng_seed: int = 123,
    theta_exact: float = 0.3,
):
    bin_data = np.load(binary_npz_path, allow_pickle=True)
    X_bool = bin_data["X"].astype(bool)
    model_ids_all = bin_data["model_ids"]

    mh = np.load(minhash_npz_path, allow_pickle=True)
    signatures = mh["signatures"]          # (m, n)
    m = signatures.shape[0]

    # sweep all exact (b,r) with b*r = m
    cfgs = all_band_configs(m)            # list of (t_imp, b, r), sorted by t_imp
    K = len(cfgs)

    rng = np.random.default_rng(rng_seed)

    F1_mat = np.zeros((n_bootstrap, K), dtype=float)
    frac_mat = np.zeros((n_bootstrap, K), dtype=float)

    eff = 0
    for _ in range(n_bootstrap):
        n = X_bool.shape[0]
        in_bag = rng.integers(0, n, size=n)
        subset_idx = np.unique(in_bag)
        if subset_idx.size < 2:
            continue

        # subset views (fixed for this bootstrap)
        sig_sub = signatures[:, subset_idx]     # (m, n_sub)
        X_sub   = X_bool[subset_idx]            # (n_sub, p)
        mid_sub = model_ids_all[subset_idx]     # (n_sub,)
        n_sub   = subset_idx.size

        for k, (t_imp, b, r) in enumerate(cfgs):
            # candidates = bucket collisions only
            pairs = lsh_candidate_pairs(sig_sub, bands=b, rows_per_band=r)

            if pairs.shape[0] == 0:
                F1_mat[eff, k] = 0.0
                frac_mat[eff, k] = 0.0
                continue

            frac_mat[eff, k] = pairs.shape[0] / (n_sub * (n_sub - 1) // 2)

            i_loc = pairs[:, 0]
            j_loc = pairs[:, 1]
            sims_exact = exact_jaccard_on_candidates(X_sub, i_loc, j_loc)

            labels_sub = cluster_with_threshold(
                n_items=n_sub,
                i_idx=i_loc,
                j_idx=j_loc,
                sims=sims_exact,
                theta=theta_exact,
            )

            stats = pairwise_confusion_and_f1_subset(
                labels=labels_sub,
                model_ids=mid_sub,
                subset_idx=np.arange(n_sub),
            )
            F1_mat[eff, k] = float(stats["F1"])

        eff += 1

    if eff == 0:
        raise RuntimeError("No effective bootstraps.")

    F1_mean = F1_mat[:eff].mean(axis=0)
    frac_mean = frac_mat[:eff].mean(axis=0)

    # sort by mean fraction
    order = np.argsort(frac_mean)

    return {
        "t_values": np.array([cfgs[i][0] for i in order]),   # implied thresholds (for labels only)
        "F1_mean": F1_mean[order],
        "frac_mean": frac_mean[order],
        "n_effective_bootstrap": eff,
        "cfgs_sorted": [cfgs[i] for i in order],            # (t_imp,b,r) per point
    }


def plot_finalF1_vs_fraction(curve: dict):
    plt.figure()
    plt.plot(
        curve["frac_mean"],
        curve["F1_mean"],
        color="black",
        linewidth=2.5,
        linestyle="-",
        marker=None,
    )
    plt.xlabel("Fraction of comparisons")
    plt.ylabel("F1-measure")
    plt.grid(False)
    plt.title(None)
    plt.show()



if __name__ == "__main__":
    curve = paper_style_curve_finalF1_vs_fraction(
    binary_npz_path=BINARY_NPZ_PATH,
    minhash_npz_path=MINHASH_NPZ_PATH,
    n_bootstrap=5,
    rng_seed=123,
    theta_exact=0.3,
)
    print("effective bootstraps:", curve["n_effective_bootstrap"])
    plot_finalF1_vs_fraction(curve)


KeyboardInterrupt: 